In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#UV
import urllib
import gzip
import json
import requests
import mysql.connector
import datetime
import threading, time
import numpy as np #python陣列的套件，命名為np
from datetime import datetime
from threading import Thread
from urllib.request import urlretrieve

def func():
    global tStart #開始爬蟲時間
    global tEnd   #結束爬蟲時間
    global timenow#當前時間
    
    print ("crawling web page")
    urlretrieve = "http://opendata.epa.gov.tw/webapi/Data/UV/?$select=County,PublishTime,SiteName,UVI&$orderby=SiteName%20desc&$skip=0&$top=1000&format=json"
    #環保署開放資料，篩選需要的欄位後呈現在網頁上
    data = requests.get(urlretrieve).json() #解析網頁內容後將json格式轉換成list
    data = str(data)
    data = data.split(",") #字串分割
    data = data[24]+","+data[25]+","+data[26]+","+data[27] #取出需要的值存入變數
    data = "{'retVal':{'0001':"+data+"}}" #增加字串讓值像是json格式
    d={}
    data = data.split("'") #字串分割
    #把值寫入字典
    d['County']=data[7]
    d['PublishTime']=data[11]
    d['SiteName']=data[15]
    d['UVI']=data[19]
    #紫外線評級寫入變數
    UVIlevel=float(data[19])
    if UVIlevel>=0.00 and UVIlevel < 3.00 :
        UVIlevel = str(UVIlevel)
        UVIlevel = '低量級'
    elif UVIlevel >= 3.00 and UVIlevel < 6.00 :
        UVIlevel = str(UVIlevel)
        UVIlevel = '中量級'
    elif UVIlevel >= 6.00 and UVIlevel < 8.00 :
        UVIlevel = str(UVIlevel)
        UVIlevel = '高量級'
    elif UVIlevel >= 8.00 and UVIlevel < 11.00 :
        UVIlevel = str(UVIlevel)
        UVIlevel = '過量級'
    elif UVIlevel >= 11.00 :
        UVIlevel = str(UVIlevel)
        UVIlevel = '危險級'
    else:
        UVIlevel = str(UVIlevel)
        UVIlevel = '無資料'
    d['UVIlevel']=UVIlevel
    print(type(d))
    #print(d)
    #print(data)
    '''
    #政府紫外線評級標準
    <3低量級
    指數3~5中量級
    指數6~7高量級
    指數8~10過量級
    指數11以上危險級
    '''
        
    timenow = time.localtime(time.time())
    print(timenow)
    a=np.array(time.localtime(time.time()))#將時間格式轉換成陣列
    date = datetime.now().strftime('%Y_%m_%d')#取出當前時間 年_月_日
    
    if a[1]==10:#當時間來到10月份
        conn = mysql.connector.connect(host="localhost", user="root", passwd="", db="UV_10", charset = 'utf8')#更換存取的db
        c = conn.cursor()
    elif a[1]==11:#當時間來到11月份
        conn = mysql.connector.connect(host="localhost", user="root", passwd="", db="UV_11", charset = 'utf8')#更換存取的db
        c = conn.cursor()
    elif a[1]==12:#當時間來到12月份
        conn = mysql.connector.connect(host="localhost", user="root", passwd="", db="UV_12", charset = 'utf8')#更換存取的db
        c = conn.cursor()
    else:
        print("db not found")
        pass
    
    if True:
        print("1")
        sql="CREATE TABLE IF NOT EXISTS " + date + "(`County` VARCHAR(4) CHARACTER SET utf8 NOT NULL , `PublishTime` DATETIME NOT NULL , `SiteName` VARCHAR(4) CHARACTER SET utf8 NOT NULL , `UVI` FLOAT(4) NOT NULL , `UVIlevel` VARCHAR(4) CHARACTER SET utf8 NOT NULL )ENGINE = InnoDB DEFAULT CHARSET=latin1 AUTO_INCREMENT=0;"
        #date以日期名稱命名table
        print(sql)
        try:
            c.execute(sql) #創建table
        except Exception as e:
            print("Mysql Error %d: %s" % (e.args[0], e.args[1]))
            pass
        if True:
            #將字典的值放入變數
            County = d['County']
            PublishTime = d['PublishTime']
            SiteName = d['SiteName']
            UVI = d['UVI']
            UVIlevel = d['UVIlevel']
            sql = "INSERT INTO " + date + " (County,PublishTime,SiteName,UVI,UVIlevel) VALUES(%s,%s,%s,%s,%s)"
            #資料寫入table
            try:
                c.execute(sql,(County,PublishTime,SiteName,UVI,UVIlevel))

            except Exception as e:
                print("Mysql Error %d: %s" % (e.args[0], e.args[1]))
                pass
        conn.commit()
        conn.close()

def start():
    t = Thread(target=func) #跑func這個function
    t.start()
   
if __name__ == '__main__':
    counter=0
    a=5
    tStart1 = time.time()
    start() #跑start這個function
    tEnd1 = time.time()
    time.sleep(1200-(tEnd1 - tStart1)) #設定每1200秒起來檢查一次
    tEnd1 = time.time()
    tstart0=tStart1
    
    while(a>0):
        tStart1 = tEnd1
        
        if((tStart1-tstart0)>=3600): #如果到3600秒起來爬資料(原始資料更新頻率)
            counter=counter+1
        else:
            tstart0=tEnd1
            counter=counter+1
            
        if(counter%3==0):
            start()
        else:
            print("")    
            
        tEnd1 = time.time()
        time.sleep(1200-(tEnd1 - tStart1))        
        tEnd1 = time.time()